In [2]:
## Importing Libraries
import os 
import pandas as pd
import xlrd
import re
from tqdm import tqdm
from datetime import datetime, timedelta

In [226]:
## Directory where all files are present 
Download_Dir=os.path.join(os.getcwd(),"Downloads")
print(Download_Dir)

C:\Users\bhara\Assignment_GNA\Downloads


In [227]:
## Function to Create DateTimeIndex
def create_datetime(time_desc,file_date):
    # if pd.isna(time_desc) or file_date is None:
    #     return pd.NaT 
    try:
        end_time_str = time_desc.split('-')[0]
        datetime_str = f"{file_date} {end_time_str}"
        return pd.to_datetime(datetime_str, format='%Y-%m-%d %H:%M')
    except Exception as e:
        print(f"Error converting time_desc '{time_desc}': {e}")



## Function to Format the data of File
def Clean_File(File_Name,File_Path):
    df_list = pd.read_html(File_Path)
    df = df_list[0]
    df = df[:-4] ## Removing Stats Rows from Dataframe
    df = df.iloc[:, 1:] ## Removing Index Column
    ## Extracting Date from the File_Name
    match = re.search(r'\d{2}-\d{2}-\d{4}', File_Name)
    if match:
        file_date_str = match.group(0)
        file_date = pd.to_datetime(file_date_str, format='%d-%m-%Y').date()
    else:
        print("Could not extract date from filename.")
        file_date = None
    # df.index = df['TimeDesc'].apply(create_datetime,file_date)
    df.index = df['TimeDesc'].apply(lambda x: create_datetime(x, file_date))
    df = df.iloc[:, 1:]
    return df

## Function That will create a csv file (if not present) or append the df into a csv file
def save_or_append_csv(df, file_path):
   
    if not os.path.exists(file_path):
        # File doesn't exist; write with header
        df.to_csv(file_path, mode='w', index=True)
        # print("Created File")
    else:
        # File exists; append without header
        df.to_csv(file_path, mode='a', header=False, index=True)
        # print("Appended data")




In [228]:
## Date Ranges
start_date = datetime(2020, 5, 17)
end_date = datetime(2025, 5, 16)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

In [229]:
## Merging All Files 
for date in tqdm(date_range,"Processing : "):
    day = date.strftime("%d")
    month = date.strftime("%m")
    year = date.strftime("%Y")
    File_Name='Report-Summary-'+str(day)+'-'+str(month)+'-'+str(year)+'.xls'
    File_Path=os.path.join(Download_Dir,File_Name)
    if os.path.exists(File_Path):    
        df=Clean_File(File_Name,File_Path)
        save_or_append_csv(df, 'Final.csv')
    else:
        print("File Not Found :- ",File_Path)

Processing : 100%|█████████████████████████████████████████████████████████████████| 1826/1826 [02:12<00:00, 13.75it/s]


In [16]:
df = pd.read_csv('Final.csv',index_col=0)
df=df[['Total']]
df.to_csv('Discom_Drawal_Total.csv')